# Creating SnowModel-LG combined numpy array 1 Jan 2010 - 31 Dec 2020

In [1]:
import numpy as np
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
import datetime
from scipy.interpolate import griddata
import tqdm
from regrid import regrid
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [2]:
#import lon, lat & snod values from original SM dataset
datapath = '/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/'
file = datapath+'SM_snod_MERRA2_01Aug1980-31Jul2018_v01.nc'
data = Dataset(file,'r')

lon = data['x']
lat = data['y']
snod = data['snod'][-3134:] # 1/1/2010 - 31/12/2017 only

snod_orig = np.array(snod) ; lon = np.array(lon) ; lat = np.array(lat)
snod_orig[snod_orig==-9999] = np.nan

#convert coordinates 
m = Basemap(projection='npstere',boundinglat=60,lon_0=0, resolution='l',round=True)
mymask = Dataset(datapath+'mask.nc')
ease_lons = np.array(mymask['lon'])
ease_lats = np.array(mymask['lat'])
x,y = m(ease_lons,ease_lats)

# create list of dates
hours = np.array(data['time'])
datess = [datetime.date(1,1,1)+datetime.timedelta(hours=np.int(hours))-datetime.timedelta(days=2) for hours in hours]
dates_orig = datess[-3134:]

#import lon, lat & snod values from SM extension
datapath = '/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/'
file = datapath+'snod_ext.nc'
data = Dataset(file,'r')
snod_ext = data['snod']

snod_ext = np.array(snod_ext)
snod_ext[snod_ext==-9999] = np.nan
days = np.arange(0,731)
dates_ext = [datetime.date(2018,8,1)+datetime.timedelta(days=np.int(days)) for days in days]

#create empty array for 1 Aug 2020 onwards
snod_missing = []
snow_missings = np.zeros((361,361))
snow_missings[:] = np.nan
for i in range(153):
    snod_missing.append(snow_missings)
days = np.arange(0,153)
dates_missing = [datetime.date(2020,8,1)+datetime.timedelta(days=np.int(days)) for days in days]
snod_missing = np.array(snod_missing)
snod_missing.shape

# combine SM_orig and SM_ext and SM_missing
snow = np.concatenate((snod_orig,snod_ext,snod_missing),axis=0)
dates = [] 
dates.append(dates_orig)
dates.append(dates_ext)
dates.append(dates_missing)

#save to file that we can easily load and use
#np.save('/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/SM_snod_combined.npy',snow)

In [3]:
# change Will's 161 x 161 grid to 160 x 160
#new_x_WG = np.full((160,160),np.nan)
#import itertools
#for i, j in tqdm.tqdm(itertools.product(np.arange(160),
#                              np.arange(160))):
#    new_x_WG[i,j]= np.mean(x_WG[i:i+1,j:j+1])
#    
#new_y_WG = np.full((160,160),np.nan)   
#for i, j in tqdm.tqdm(itertools.product(np.arange(160),
#                          np.arange(160))):
#    new_y_WG[i,j]= np.mean(y_WG[i:i+1,j:j+1])
#np.save('/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/new_x_WG.npy',new_x_WG)
#np.save('/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/new_y_WG.npy',new_y_WG)

In [4]:
#regrid onto will grid and save
snow = np.load('/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/SM_snod_combined.npy')
mymask = Dataset('/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/mask.nc')
SM_lon = np.array(mymask['lon'])
SM_lat = np.array(mymask['lat'])
lats = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lat.npy')
lons = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lon.npy')

In [5]:
days = np.arange(0,4018)
snows = np.full((4018,160,160), np.nan)
for day in tqdm.tqdm(days):
    snod = snow[day]
    SM_new = regrid(snod, SM_lon, SM_lat, lons, lats,'nearest')
    snows[day] = SM_new
np.save('/Users/carmennab/PhD/OI_PolarSnow/SnowModelLG/SM_snod_combined_regridded.npy',snows)

100%|███████████████████████████████████████| 4018/4018 [08:56<00:00,  7.48it/s]
